# 前準備

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# make deterministic
from mingpt.utils import set_seed
set_seed(44)

In [ ]:
import os
import math
import time
from tqdm import tqdm
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hugging Faceからロードするためのインポート例
from datasets import load_dataset

# mingpt関連
from mingpt.dataset import CharDataset
from mingpt.utils import sample
from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig
import os
import math
import time
from tqdm import tqdm
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from torch.nn import functional as F
from data import get_othello
from data.sudoku import permit, start_hands, OthelloBoardState, permit_reverse
from mingpt.dataset import CharDataset
from mingpt.utils import sample
from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig

# データセットのロード

In [ ]:
# ここで "huggingface_sudoku" は仮の名前。
# 実際には該当のデータセット名に置き換えてください。
ds = load_dataset("huggingface_sudoku", split="train")

# ds はデータの各サンプルを辞書形式で持っていることを想定します。
# 例: {"id": ..., "sequence": [...]}  のように手順が配列で格納されていると仮定。
# sequence をGPT用のトークン列に変換するには、CharDatasetなどで処理できる形式へ合わせる。
# 一例として、すべてのサンプルをまとめたリスト (list of lists) にする:
all_sequences = []
for example in ds:
    # example["sequence"] がすでに int のトークン列になっている想定
    all_sequences.append(example["sequence"])

# CharDatasetに渡す
train_dataset = CharDataset(all_sequences)

# モデルの設定

In [ ]:
mconf = GPTConfig(
    vocab_size=train_dataset.vocab_size,
    block_size=train_dataset.block_size,  # 例: 81 (またはさらに大きい場合も)
    n_layer=8,
    n_head=8,
    n_embd=512
)
model = GPT(mconf)

# トレーナーの設定

In [ ]:
max_epochs = 250
# initialize a trainer instance and kick off training
t_start = time.strftime("_%Y%m%d_%H%M%S")
tconf = TrainerConfig(
    max_epochs=max_epochs, 
    batch_size=512*8,  # assuming 8 GPU's
    learning_rate=5e-4,
    lr_decay=True, 
    warmup_tokens=len(train_dataset)*train_dataset.block_size*5, 
    final_tokens=len(train_dataset)*train_dataset.block_size*max_epochs,
    num_workers=0, 
    ckpt_path=f"./ckpts/gpt_sudoku_at{t_start}.ckpt", 
)
trainer = Trainer(model, train_dataset, None, tconf)
device = trainer.device
print(t_start)

# トレーニング

In [ ]:
trainer.train()

# 合法手か検証

In [ ]:
if not synthetic_or_championship:  # for GPT trained on both datasets, use the validation set of synthetic for validation
    othello = get_othello(ood_num=-1, data_root=None, wthor=True)

In [ ]:
total_nodes = 0
success_nodes = 0

bar = tqdm(othello.val[:1000])
for whole_game in bar:
    length_of_whole_game = len(whole_game)
    for length_of_partial_game in range(1, length_of_whole_game):
        total_nodes += 1
        context = whole_game[:length_of_partial_game]
        x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None, ...].to(device)
        y = sample(model, x, 1, temperature=1.0)[0]
        completion = [train_dataset.itos[int(i)] for i in y if i != -1]
        try:
            OthelloBoardState().update(completion, prt=False)
        except Exception:
#             fail_nodes.append([permit_reverse(_) for _ in context])
            pass
        else:
            success_nodes += 1
    bar.set_description(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")
print(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")

In [ ]:
1 - success_nodes/total_nodes